In [1]:
import numpy as np 
from PIL import Image 
from scipy.ndimage import filters
import pylab as pylab
from scipy.ndimage import measurements ,morphology
from scipy.ndimage import filters
from PIL import Image, ImageOps
from pylab import *
import os
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
# Открыть и обрезать изображение
def open_and_cut_image(path):
    im = Image.fromarray(np.array(Image.open(path)))
    border = (25, 120, 25, 110)  # left, up, right, bottom
    im = ImageOps.crop(im, border)
    return im


def get_feet_lables(im):
    # делаю изображение серым
    im = im.convert('L')
    # Применяю фильтр Гаусса
    im = Image.fromarray(filters.gaussian_filter(im, 10))
    # Реверс изображения
    im = 255 - np.array(im)
    # Применяю пороговый фильтр и перевожу в массив
    im = Image.fromarray(1 * (im < 242))
    # Получаю количество объектов и их расположения
    labels, nbr_obgjects = measurements.label(im)
    return labels, nbr_obgjects


def get_isolatet_photos_of_foots(im, nbr_obgjects):
    # Перевожу изображение в массив
    im = np.array(im)

    if nbr_obgjects > 10:
        shape_ = list(im.shape)
        shape_.insert(0, 10)
        array_of_images = np.zeros(shape=shape_)
    else:
        shape_ = list(im.shape)
        shape_.insert(0, nbr_obgjects)
        array_of_images = np.zeros(shape=shape_)

    # Перебираю массив по высоте
    for y in range(0, im.shape[0]):
        # По ширине
        for x in range(0, im.shape[1]):
            # Создаю пустой пиксель
            pixel = []
            # Если лайбл больше 0 - беру его из изображения и добавляю в лист в его лейбл
            if labels[y][x] > 0 and labels[y][x] < 11:
                num_of_lable = labels[y][x]
                pixel = list(im[y][x])
                pixel = [pixel[0], pixel[1], pixel[2]]
                array_of_images[(num_of_lable - 1)][y][x] = pixel
    return array_of_images


def save_list_of_labels(list_of_labels, save_path, pefics_of_im):
    # Перевожу массивы в фотографии
    num = 1
    if os.path.exists(save_path):
        pass
    else:
        os.mkdir(save_path)
    for im in list_of_labels:
        new_im = (im).astype(np.uint8)
        new_im = Image.fromarray(new_im)
        image = new_im.save(f"{save_path}/{pefics_of_im}_{num}.png")
        num += 1

In [8]:
def load_model(model_path): 
    model = tf.keras.models.load_model(model_path)
    return model


def get_result(impath):
    impath = 'C:/for_learn/for_pred/1/a940dea7-ce38-4cf9-bfcf-cf356d71bf8b.jpg_1.png'
    img = tf.keras.utils.load_img(impath,
                                  target_size=(256, 256))
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create a batch
    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    print(
        ("This image (impath) most likely belongs to {} with a {:.2f} percent confidence.".
         format(class_names[np.argmax(score)], 100 * np.max(score))))

In [10]:
dirname = 'datasets/Исходные данные'
files = os.listdir(dirname)
save_path = 'datasets/Примеры предобработки'
for file_name in files:
    path = dirname + '/' + file_name
    # Открываю изображение
    im = open_and_cut_image(path)
    # Получаю лейблы и количество объектов
    labels, nbr_obgjects = get_feet_lables(im)
    list_of_labels = get_isolatet_photos_of_foots(im, nbr_obgjects)
    # Сохраняю в файл
    save_list_of_labels(list_of_labels, save_path, file_name)
model = load_model('saved_model/my_model')
class_names = ['0', '1', '2', '3']

In [11]:
files = os.listdir(save_path)
for file_name in files:
    impath = dirname + '/' + file_name
    get_result(impath)

This image (impath) most likely belongs to 3 with a 100.00 percent confidence.
This image (impath) most likely belongs to 3 with a 100.00 percent confidence.
This image (impath) most likely belongs to 3 with a 100.00 percent confidence.
This image (impath) most likely belongs to 3 with a 100.00 percent confidence.
This image (impath) most likely belongs to 3 with a 100.00 percent confidence.
This image (impath) most likely belongs to 3 with a 100.00 percent confidence.
This image (impath) most likely belongs to 3 with a 100.00 percent confidence.
This image (impath) most likely belongs to 3 with a 100.00 percent confidence.
This image (impath) most likely belongs to 3 with a 100.00 percent confidence.
This image (impath) most likely belongs to 3 with a 100.00 percent confidence.
This image (impath) most likely belongs to 3 with a 100.00 percent confidence.
This image (impath) most likely belongs to 3 with a 100.00 percent confidence.
This image (impath) most likely belongs to 3 with a 

# Спасибо за проверку!